In [14]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from utilsforecast.plotting import plot_series
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [6]:


df = pd.read_csv('./data/Clean_training.csv')

df['unique_id'] = df['Store'].astype(str) + '_' + df['Dept'].astype(str)
df = df.rename(columns={'Date': 'ds', 'Target': 'y'})
df['ds'] = pd.to_datetime(df['ds'])
df['IsHoliday'] = df['IsHoliday'].astype(int)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
df[markdown_cols] = df[markdown_cols].fillna(0)
df = df.sort_values(['unique_id', 'ds'])

largest_series_id = df['unique_id'].value_counts().idxmax()
df_subset = df[df['unique_id'] == largest_series_id].copy()

print(f"Training model on the largest series: {largest_series_id}")

horizon = 12
input_size = 2 * horizon

train_df = df_subset[:-horizon]
test_df = df_subset

model = NBEATSx(h=horizon,
                input_size=input_size,
                max_steps=500,
                scaler_type='standard',
                learning_rate=0.005,
                random_seed=42)

nf = NeuralForecast(models=[model], freq='W')
nf.fit(df=train_df)

Y_hat_df = nf.predict()

fig = plot_series(test_df, Y_hat_df, max_insample_length=150, plot_random=False)
plt.tight_layout()
fig.savefig('nbeatsx_forecast_with_test.png')
print("\nSuccess! Forecast plot saved as nbeatsx_forecast_with_test.png")

forecasts_df = pd.merge(test_df, Y_hat_df, on=['unique_id', 'ds'])


y_true = test_df['y'].tail(horizon).values

y_pred = Y_hat_df['NBEATSx'].values

mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
mape = mean_absolute_percentage_error(y_true, y_pred)

print("\n--- Model Performance Metrics ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2%}")
print("---------------------------------")

Seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.4 M     Trainable params
900       Non-trainable params
2.4 M     Total params
9.789     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Training model on the largest series: 9_96


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]


Success! Forecast plot saved as nbeatsx_forecast_with_test.png

--- Model Performance Metrics ---
Mean Absolute Error (MAE): 262.14
Mean Squared Error (MSE): 155870.00
Mean Absolute Percentage Error (MAPE): 6.50%
---------------------------------


<Figure size 640x480 with 0 Axes>